In [1]:
from random import seed
import numpy as np
import pandas as pd
import json
import os
import sys
import gc
import shutil
from pprint import pprint
from pathlib import Path
from typing import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import categorical_accuracy

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler

import sys

from matplotlib import pyplot as plt

In [2]:
sys.path.append(str(Path().resolve().parent))

from src.utils import seed_every_thing, Config, plot_metric, reduce_tf_gpu_memory, reduce_mem_usage, fetch_custom_data

In [3]:
logdir = Path().resolve().parent / "logs"
datadir = Path().resolve().parent / "data"
cachedir = Path().resolve().parent / "cache"

In [4]:
cfg_file_path = logdir / "lstm-less-addfeatures-cut-large-custom-mae" / "config.json"
with open(cfg_file_path, "rb") as f:
    config = json.load(f)

In [5]:
def build_model(config: Config, n_features) -> keras.models.Sequential:
    model = keras.models.Sequential([keras.layers.Input(shape=(config.cut, n_features))])
    for n_unit in config.n_units:
        model.add(
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    n_unit,
                    return_sequences=True,
                )
            )
        )
    for n_unit in config.n_dense_units:
        model.add(keras.layers.Dense(n_unit, activation="selu"))
    model.add(keras.layers.Dense(1))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config.lr), loss='mae')
    return model

def compute_metric(df, merge=True):
    y_true, y_pred, u_out = df["pressure"].values, df["pred"].values, df["u_out"].values
    w = (1 - u_out)
    
    if merge:
        mae = w * np.abs(y_true - y_pred)
        mae = np.sum(mae) / np.sum(w)
        return mae
    else:
        y_true, y_pred, w = y_true.reshape(-1, 35), y_pred.reshape(-1, 35), w.reshape(-1, 35)
        maes = []
        for yt, yp, w_ in zip(y_true, y_pred, w):
            mae = w_ * np.abs(yt - yp)
            mae = np.sum(mae) / np.sum(w_)
            maes.append(mae)
        return np.array(maes)

In [6]:
config = Config().update(config)
reduce_tf_gpu_memory(gpu_id=config.gpu_id)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU') memory growth: True
PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU') memory growth: True


In [7]:
DEBUG = False
train_df = reduce_mem_usage(pd.read_csv(cachedir / f"train_lstm-less-cut-addfeatures_debug{DEBUG}.csv"))
test_df = reduce_mem_usage(pd.read_csv(cachedir / f"test_lstm-less-cut-addfeatures_debug{DEBUG}.csv"))
kfolds = train_df.iloc[0 :: config.cut]["kfold"].values

Memory usage of dataframe is 2216.21 MB
Memory usage after optimization is: 506.20 MB
Decreased by 77.2%
Memory usage of dataframe is 1464.04 MB
Memory usage after optimization is: 334.11 MB
Decreased by 77.2%


In [8]:
features = list(train_df.drop(["kfold", "pressure"], axis=1).columns)
cont_features = [
    f for f in features if ("RC_" not in f) and ("R_" not in f) and ("C_" not in f) and ("u_out" not in f)
]

In [9]:
RS = RobustScaler()
train_df[cont_features] = RS.fit_transform(train_df[cont_features])
test_df[cont_features] = RS.transform(test_df[cont_features])
train_data, test_data = train_df[features].values, test_df[features].values

train_data = train_data.reshape(-1, config.cut, train_data.shape[-1])
targets = train_df[["pressure"]].to_numpy().reshape(-1, config.cut)
test_data = test_data.reshape(-1, config.cut, test_data.shape[-1])

In [10]:
def inference():
    with tf.device(f"/GPU:2"):
        latent_preds = np.empty((train_data.shape[0], train_data.shape[1], config.n_dense_units[-1])).astype(np.float32)
        test_latent_preds = []

        for fold in range(config.n_splits):
            train_idx, test_idx = (kfolds != fold), (kfolds == fold)
            print("-" * 15, ">", f"Fold {fold+1}", "<", "-" * 15)
            savedir = logdir / "lstm-less-addfeatures-cut-large-custom-mae"/ f"fold{fold}"

            X_train, X_valid = train_data[train_idx], train_data[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = build_model(config=config, n_features=len(features))
            model.load_weights(savedir / "weights_custom_best.h5")
            latent_features = model.layers[-2].output
            new_model = keras.models.Model(inputs=model.inputs, outputs=latent_features)   

            latent_preds[test_idx, :, :] = new_model.predict(X_valid)
            test_latent_preds.append(new_model.predict(test_data))

            del model, new_model, X_train, X_valid, y_train, y_valid
            keras.backend.clear_session()
            gc.collect()
    return latent_preds, test_latent_preds

In [11]:
latent_preds, test_latent_preds = inference()

2021-10-30 14:27:30.492988: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-30 14:27:31.986695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21746 MB memory:  -> device: 0, name: NVIDIA Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0
2021-10-30 14:27:31.987926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30538 MB memory:  -> device: 1, name: NVIDIA Tesla V100-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2021-10-30 14:27:31.988977: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/r

--------------- > Fold 1 < ---------------


2021-10-30 14:27:34.822689: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-30 14:27:36.805048: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


--------------- > Fold 2 < ---------------
--------------- > Fold 3 < ---------------
--------------- > Fold 4 < ---------------
--------------- > Fold 5 < ---------------
--------------- > Fold 6 < ---------------
--------------- > Fold 7 < ---------------
--------------- > Fold 8 < ---------------
--------------- > Fold 9 < ---------------
--------------- > Fold 10 < ---------------


In [12]:
savedir = logdir / "lstm-less-addfeatures-cut-large-custom-mae"

np.save(str(savedir / "valid_latent_feature"), latent_preds)
np.savez(str(savedir / "test_latent_feature"), *test_latent_preds)